In [3]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)


In [ ]:
# View all of the classes that automap found
class_names = Base.classes.keys()
for class_name in class_names:
    print(class_name)

In [ ]:
# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)


# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
most_recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)


# Perform a query to retrieve the data and precipitation scores

year_results = (session.query(Measurement.date, func.max(Measurement.prcp))
                 .filter(func.strftime('%Y-%m-%d', Measurement.date) > one_year_ago)
                 .group_by(Measurement.date)
                 .all())
# Save the query results as a Pandas DataFrame. Explicitly set the column names
pcrp_df = pd.DataFrame(year_results, columns=['date', 'precipitation'])
pcrp_df.set_index('date', inplace=True)

# Sort the dataframe by date
pcrp_df.sort_values('date')

#plot the results of the query
pcrp_df.plot().set(xlabel='Date', ylabel='Inches')
plt.xticks(rotation=45)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=1)

# Save the plot as a PNG image in the "images" folder
folder_name = "Images"
os.makedirs(folder_name, exist_ok=True)  # Create the folder if it doesn't exist
file_path = os.path.join(folder_name, "pcrp_plot.png")
plt.savefig(file_path, format='png')





In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
stats_df = pd.DataFrame(pcrp_df['precipitation'].describe())
stats_df

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_number = session.query(Station.station).count()
station_number

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations=session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = active_stations[0][0] 

# Calculate the lowest, highest, and average temperature for the most active station
temperature_stats = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)) \
                           .filter(Measurement.station == most_active_station) \
                           .all()

lowest_temp = temperature_stats[0][0]
highest_temp = temperature_stats[0][1]
avg_temp = temperature_stats[0][2]

lowest_temp, highest_temp, avg_temp

In [ ]:

# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temperature_data = session.query(Measurement.tobs) \
                          .filter(Measurement.station == most_active_station) \
                          .filter(Measurement.date >= one_year_ago) \
                          .all()

# Convert the query result to a DataFrame
temperature_df = pd.DataFrame(temperature_data, columns=['tobs'])

# Plot the temperature data as a histogram
temperature_df.plot(kind='hist', bins=12).set(xlabel='Temperature', ylabel='Frequency')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=1)

# Save the plot as a PNG image in the "images" folder
folder_name = "Images"
os.makedirs(folder_name, exist_ok=True)  # Create the folder if it doesn't exist
file_path = os.path.join(folder_name, "station_plot.png")
plt.savefig(file_path, format='png')

# Close session

In [ ]:
# Close Session
session.close()